In [12]:
#Questions: Are the different coefficients uniform amongst all the nodes in the graph ? Right now, they are equal for all nodes. 
# Yes they should be equal since we are assuming that the effect is shared and we just variate over the value of the feature.
#To easily compute and update the happiness, we keep the following properties true during the evolution of the graph:
# - the attribute contains the happiness of the current node at each step
# - at the begining of a step, the attribute 'happiness_updating' contains coeff_X*X + coeff_Z*Z + coeff_previous_happiness*happiness.
#Therefore we only have to add the influence of all the other nodes at 'happiness_updating' to obtain the updated happiness.

#Here, at each step, we choose a random node and add one friend to it. Then update the happiness of everyone.

In [13]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy
import random
import math
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as sm

In [14]:
#Defining our global variables:
N_nodes = 100

In [15]:
#Defining graph
G=nx.DiGraph()
G.add_nodes_from(range(N_nodes))

In [16]:
#Defining attributes
for i in range(N_nodes):
    G.node[i]['X'] = random.random()
    G.node[i]['Z'] = random.random()
    G.node[i]['coeff_X'] = 1
    G.node[i]['coeff_Z'] = 1
    G.node[i]['coeff_previous_happiness'] = 1
    G.node[i]['coeff_influence'] = 1
    G.node[i]['happiness'] = G.node[i]['X'] * G.node[i]['coeff_X'] + G.node[i]['Z'] * G.node[i]['coeff_Z']
    G.node[i]['happiness_updating'] = G.node[i]['X'] * G.node[i]['coeff_X'] + G.node[i]['Z'] * G.node[i]['coeff_Z']+ G.node[i]['coeff_previous_happiness']* G.node[i]['happiness']

In [17]:
#Creating similarity matrices
similarity_matrix = [[0 for i in range(N_nodes)] for j in range(N_nodes)]
observable_matrix = [[0 for i in range(N_nodes)] for j in range(N_nodes)]
similarity_coefficient = 3
for i in range(N_nodes):
    for j in range(i):
        similarity_matrix[i][j] = -similarity_coefficient*math.sqrt((G.node[i]['X'] - G.node[j]['X'])**2 + (G.node[i]['Z'] - G.node[j]['Z'])**2)
        observable_matrix[i][j] = math.sqrt((G.node[i]['X'] - G.node[j]['X'])**2) 
        similarity_matrix[i][j] = 1.0 / (1.0 + math.exp(-similarity_matrix[i][j]))
        similarity_matrix[j][i] = similarity_matrix[i][j]
        observable_matrix[j][i] = observable_matrix[i][j]
        


In [18]:
#Creating set of potential friends for each node
def create_list_potential_friends(Graph):
    list_potential_friends = [[] for i in range(N_nodes)]
    for i in range(N_nodes):
        for j in range(N_nodes):
            if i != j and (i,j) not in Graph.edges():
                if random.random() < similarity_matrix[i][j]:
                    list_potential_friends[i].append(j)
    return list_potential_friends

list_potential_friends

In [19]:
#Function to add to the node "node" a random friend from his list of friends (and to remove him from the list)
def add_one_friend(node, Graph):
    if len(list_potential_friends[node])>0:
        new_friend = random.choice(list_potential_friends[node])
        list_potential_friends[node].remove(new_friend)
        Graph.add_edge(node, new_friend)

In [20]:
#This step is made to add a friend for each node of the Graph
def init_step(Graph):
    for node in range(N_nodes):
        add_one_friend(node, Graph)

In [21]:
#Update happiness 
def update_happiness(Graph):
    for (a,b) in Graph.edges_iter():
        Graph.node[a]['happiness_updating'] +=  G.node[a]['coeff_influence'] *Graph.node[b]['happiness']/len(Graph.neighbors(a))
    for i in range(N_nodes):
        Graph.node[i]['happiness'] = G.node[i]['happiness_updating']
        Graph.node[i]['happiness_updating'] = G.node[i]['coeff_previous_happiness']*G.node[i]['happiness'] + G.node[i]['X'] * G.node[i]['coeff_X'] + G.node[i]['Z'] * G.node[i]['coeff_Z']

In [22]:
def run_step(Graph):
    for node in range(N_nodes):
        add_one_friend(node, Graph)
        update_happiness(Graph)

In [23]:
#NO fixed point algorithm (convergence not guaranteed), rather use the outputs at the previous time.
#Think about best convergence, not go through the nodes in the same way at each step

In [24]:
list_potential_friends

NameError: name 'list_potential_friends' is not defined

In [ ]:
#Main run for the graph
nb_steps=0
init_step(G)
update_happiness(G)
for i in range(nb_steps):
    run_step(G)

In [ ]:
#Plotting the last graph obtained
nx.draw(G)
plt.show()

In [ ]:
for edge in G.edges():
    print edge

print (0,2) in G.edges()

In [ ]:
# Creation of Treatment and Response Table
edges = []
treatment = []
response = []
observable = []

for j in range(N_nodes):
    observed_similarity = []
    n_friends = 0
    for i in range(N_nodes):
        if i!=j:
            if (i,j) in G.edges():
                n_friends+=1
                edges.append((i,j))
                treatment.append(True)
                response.append(G.node[i]['happiness'])
                observable.append(G.node[i]['X'])
            else:
                observed_similarity.append((observable_matrix[i][j],(i,j)))
    sorted(observed_similarity)
    for k in range(n_friends):
        control = observed_similarity[k][1]
        edges.append(control)
        treatment.append(False)
        response.append(G.node[control[0]]['happiness'])
        observable.append(G.node[control[0]]['X'])
        
data = pd.DataFrame({'Edges':edges,
                    'Treatment': treatment,
                    'Response': response,
                    'X': observable})
        
        
        

In [ ]:
data

In [ ]:
result = sm.ols(formula="Response ~ Treatment + X", data=data).fit()

In [ ]:
result.params